<a href="https://colab.research.google.com/github/FS75/ict3104-team01-2023/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🕺🕺🕺 Install Environment

In [ ]:
# !update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
# !update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
# !python --version
# !apt-get update
# !apt install software-properties-common
# !sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
# !apt-get install python3-pip

print('Git clone project and install requirements...')
!rm -rf ict3104-team01-2023 && git clone https://github.com/FS75/ict3104-team01-2023.git
# %cd ict3104-team01-2023
# !git checkout T01-27-choose-a-dataset-subfolder
# %cd ..
!rm -rf mmpose && git clone https://github.com/open-mmlab/mmpose.git

In [ ]:
!cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!export PYTHONPATH=/content/ict3104-team01-2023/FollowYourPoseTeam1:$PYTHONPATH
# %pip install -r requirements.txt
!python3.10 -m pip install -q -U --pre triton
!apt update
!python3.10 -m pip install -q diffusers==0.11.1 torch==1.13.1 transformers==4.26.0 bitsandbytes==0.35.4 \
imageio-ffmpeg xformers==0.0.16  --extra-index-url https://download.pytorch.org/whl/cu113


In [ ]:
%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!python3.10 -m pip install -r requirements.txt

## 💃💃💃 Weight

In [ ]:
%mkdir checkpoints
!git lfs install
!git clone https://huggingface.co/YueMafighting/FollowYourPose_v1
%mv /content/ict3104-team01-2023/FollowYourPoseTeam1/FollowYourPose_v1/* /content/ict3104-team01-2023/FollowYourPoseTeam1/checkpoints/
%rm -rf FollowYourPose_v1

## 🕺🕺🕺 Sample

Due to memory of GPU, we recommend set video_length=8 in ./config/pose_sample.yaml for running successfully.

Meanwhile, we should keep the skeleton frame length(./followyourpose/pipelines/pipeline_followyourpose.py:422 ) equal with video_length

In [ ]:
!pip install tensorrt

In [14]:
%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!pwd
!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="configs/pose_sample.yaml"  --skeleton_path="./pose_example/vis_ikun_pose2.mov"

100% 50/50 [03:48<00:00,  4.58s/it]
Video loaded successfully: /content/GeneratedVideos/1000-33-2023-10-30 03:17:33.980893/Astronaut on the beach.mp4
Moviepy - Building video /content/GeneratedVideos/1000-33-2023-10-30 03:17:33.980893/Astronaut on the beach_with_captions.mp4.
Moviepy - Writing video /content/GeneratedVideos/1000-33-2023-10-30 03:17:33.980893/Astronaut on the beach_with_captions.mp4

Moviepy - Done !
Moviepy - video ready /content/GeneratedVideos/1000-33-2023-10-30 03:17:33.980893/Astronaut on the beach_with_captions.mp4
I0000 00:00:1698636093.692538    6308 tfrt_cpu_pjrt_client.cc:352] TfrtCpuClient destroyed.


## 💃💃💃 The videos are output into ./checkpoints/inference/sample-xxx/<prompt>.gif

##**Data Exploration Section** 🔍🔍🔍
####This section loads, lists and displays video data from the Charades project. Here you can view the video playback of your chosen file.
-------------------
  
####How to use:
1. Click "Run Cell", the play button on the left
2. Select a video from the dropdown menu.
3. Click "Play Video"

In [45]:
# @title
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from moviepy.editor import *
from IPython.display import clear_output

!ls
def play_video(change):
    clear_output(wait=True)  # clear the current cell output before displaying new video
    if file_selector.value:
        # Split the value into folder and filename
        folder, filename = file_selector.value.split(" - ", 1)
        video_path = os.path.join('/content/ict3104-team01-2023/data/videos', folder, filename)
        video_clip = VideoFileClip(video_path)

        # Resize the video to the desired dimensions
        video_resized = video_clip.resize(newsize=(512, 512))

        # Display the video
        display(file_selector)  # re-display the dropdown
        display(play_button)  # re-display the play button
        # Display the resized video
        display(video_resized.ipython_display())
    else:
        print("Please select a video")

# Base path
base_path = '/content/ict3104-team01-2023/data/videos'

# Subfolders list
subfolders = ['charades', 'youtube']

# Collect all .mp4 files from the subfolders with headers
files = []
for subfolder in subfolders:
    subfolder_path = os.path.join(base_path, subfolder)
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)) and f.endswith('.mp4'):
            files.append(f"{subfolder} - {f}")

# Create a dropdown menu with the file names
file_selector = widgets.Dropdown(
    options=files,
    description='Select Video:',
    disabled=False,
)

# Create a button to play the video
play_button = widgets.Button(description="Play Video")

# Attach the play_video function to the button's on_click event
play_button.on_click(play_video)

# Display the dropdown menu and play button
display(file_selector)
display(play_button)

app_followyourpose.py  followyourpose		    multi_train.sh    sample.sh
app.py		       gif_results		    output	      train_followyourpose.py
checkpoints	       inference_followyourpose.py  pose_example      txt2video.py
configs		       inference_mmpose.py	    quick_demo.ipynb
example.py	       LICENSE			    README.md
example_video.zip      mmpose			    requirements.txt


Dropdown(description='Select Video:', options=('charades - 0CX32.mp4', 'charades - 0CVZN.mp4', 'charades - 0BX…

Button(description='Play Video', style=ButtonStyle())

## **Inference Section** 💡💡💡
####This section allows you to perform inference using a pretrained genAI model based on the Charades project.  
-------------------
  
####How to use (Please Add):
1.
2.
3.

### Customisation of configs (aka models)

In [7]:
pip install ruamel.yaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 13.9 MB/s eta 0:00:00


In [46]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import shutil
from collections import OrderedDict
import ruamel.yaml

yaml = ruamel.yaml.YAML()
yaml.preserve_quotes = True
yaml.indent(mapping=2, sequence=4, offset=2)

config_template = '/content/ict3104-team01-2023/FollowYourPoseTeam1/configs/pose_sample_template.yaml'

# Define a custom layout for the input widget
input_layout = widgets.Layout(width='500px')

button_layout = widgets.Layout(width='250px')

# Create input widgets with the custom layout
file_name_input = widgets.Text(
    value='',
    placeholder='Config name here (no .yaml in string)',
    description='Config name:',
    disabled=False,
    layout=input_layout
)

video_length_input = widgets.IntText(
    value=8,
    description='Video Length:',
    disabled=False,
    layout=input_layout
)

width_input = widgets.IntText(
    value=512,
    description='Width:',
    disabled=False,
    layout=input_layout
)

height_input = widgets.IntText(
    value=512,
    description='Height:',
    disabled=False,
    layout=input_layout
)

# Container to hold dynamically created prompt input fields
prompt_inputs = [
    widgets.Text(
        value='',
        placeholder='Type your prompt here',
        description='Prompt 1:',
        disabled=False,
        layout=input_layout
    )
]

# Function to handle the click of "Add Prompt" button
def add_prompt(button):
    # Create a new prompt input field
    new_prompt = widgets.Text(
        value='',
        placeholder='Type your prompt here',
        description=f'Prompt {len(prompt_inputs) + 1}:',
        disabled=False,
        layout=input_layout
    )
    prompt_inputs.append(new_prompt)

    # Redraw all UI elements
    redraw_ui()

# Create an "Add prompt" button
add_prompt_button = widgets.Button(description="Add Prompt", layout=button_layout)
add_prompt_button.on_click(add_prompt)

def remove_latest_prompt(button):
    if len(prompt_inputs) > 1:  # To ensure there's always at least one prompt
        prompt_inputs.pop()
        redraw_ui()

remove_prompt_button = widgets.Button(description="Remove Latest Prompt", layout=button_layout)
remove_prompt_button.on_click(remove_latest_prompt)

def copy_file(button):
    # Check for config name and at least one valid prompt
    if not file_name_input.value or not any(prompt.value for prompt in prompt_inputs):
        clear_output(wait=True)
        redraw_ui()
        print("Please specify a config name and at least one prompt.")
        return

    # Redraw UI
    redraw_ui()

    # Check if the template file exists
    if os.path.exists(config_template):
        with open(config_template, 'r') as f:
            content = yaml.load(f)

        # Update the yaml content with input values
        content['validation_data']['video_length'] = video_length_input.value
        content['validation_data']['width'] = width_input.value
        content['validation_data']['height'] = height_input.value
        content['validation_data']['prompts'] = [prompt.value for prompt in prompt_inputs if prompt.value]

        # Get the user's input for the file name
        dest_file_name = file_name_input.value + ".yaml"

        # Define the destination path for the copied file
        dest_path = '/content/ict3104-team01-2023/data/pre_trained_models/' + dest_file_name

        # Write the updated content to the new file
        with open(dest_path, 'w') as f:
            yaml.dump(content, f)

        print(f"YAML file {dest_file_name} generated at: {dest_path}")
    else:
        print("Template file not found!")


copy_button = widgets.Button(description="Generate Config", layout=button_layout)
copy_button.on_click(copy_file)

def redraw_ui():
    clear_output(wait=True)
    display(file_name_input, video_length_input, width_input, height_input)
    for prompt in prompt_inputs:
        display(prompt)
    display(add_prompt_button, remove_prompt_button, copy_button)

# Display the initial UI components
redraw_ui()


Text(value='pose2', description='Config name:', layout=Layout(width='500px'), placeholder='Config name here (n…

IntText(value=8, description='Video Length:', layout=Layout(width='500px'))

IntText(value=512, description='Width:', layout=Layout(width='500px'))

IntText(value=512, description='Height:', layout=Layout(width='500px'))

Text(value='The iron man, on the beach', description='Prompt 1:', layout=Layout(width='500px'), placeholder='T…

Button(description='Add Prompt', layout=Layout(width='250px'), style=ButtonStyle())

Button(description='Remove Latest Prompt', layout=Layout(width='250px'), style=ButtonStyle())

Button(description='Generate Config', layout=Layout(width='250px'), style=ButtonStyle())

YAML file pose2.yaml generated at: /content/ict3104-team01-2023/data/pre_trained_models/pose2.yaml


### Selecting a pre-trained model

In [47]:
import ipywidgets as widgets
from IPython.display import display
import os

# Assuming your folder path is '/
folder_path = '/content/ict3104-team01-2023/data/pre_trained_models/'

# List all files in the folder
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Create a dropdown menu with the file names
file_selector = widgets.Dropdown(
    options=files,
    description='Select Model:',
    disabled=False,
)

# Display the dropdown menu
display(file_selector)

# Now, the selected file can be accessed using file_selector.value

Dropdown(description='Select Model:', options=('pose1.yaml', 'default.yaml'), value='pose1.yaml')

### For Input as Skeleton


In [52]:
#@markdown Select video After running the code
import os
import ipywidgets as widgets
from IPython.display import display, HTML

# Base path
base_path = '/content/ict3104-team01-2023/data/videos'

# Subfolders list
subfolders = ['charades', 'youtube', "Cook", "Drink", "Drink2", "Drink3", "Eat", "GetupSitdown"
, "GetupSitdown2", "Readbook", "Usecomputer", "Usephone", "Usetablet", "Usetablet2", "Walk", "WatchTV"]

mp4_files = []
for subfolder in subfolders:
    subfolder_path = os.path.join(base_path, subfolder)
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)) and (f.endswith('.mp4') or f.endswith('.avi')) :
            mp4_files.append(f"{subfolder} - {f}")

dropdown = widgets.Dropdown(
    options=mp4_files,
    description='Select video:',
    disabled=False,
)
def update_link(change):
    selected_mp4_file  = dropdown.value
    link = f'<a href="{selected_mp4_file}" target="_blank">{selected_mp4_file}</a>'
    link_output.value = link

dropdown.observe(update_link, 'value')

# Display the dropdown widget
display(dropdown)

# Create an output widget to display the selected directory as a link
link_output = widgets.Output()
display(link_output)

#@markdown Enter the things to show in the video
TEXT_Prompts = "" #@param {type:"string"}


Dropdown(description='Select video:', options=('charades - 0CX32.mp4', 'charades - 0CVZN.mp4', 'charades - 0BX…

Output()

##**Training Section** 🏋🏼‍♂️🏋🏼‍♂️🏋🏼‍♂️
####This section allows you to train a new genAI model based on the Charades project.
-------------------
  
####How to use:
1. Run the first two cells below, then the third cell
  1. Take note to restart the run time before running the third cell
  2. Click on runtime from the menubar options
  3. Click on restart runtime
  4. Wait for system to complete restart before running third cell
2. Select the video you wish to use to extract a skeleton from
3. Click Generate and wait, the output cell will inform you when it has finished generating



In [ ]:
!pip install openmim
!pip install tqdm
!mim install mmengine
!mim install "mmcv>=2.0.1"
!mim install "mmdet>=3.0.0"
!rm -rf mmpose && git clone https://github.com/open-mmlab/mmpose.git

In [ ]:
import os
os.chdir('/content/mmpose')
!pip install -e .
import mmpose

###Choose a video and click Generate to start training process/generate skeleton.

<b>Note: only .mp4 videos are supported.</b>
<b>To train multiple videos at once, click the checkbox, and choose the folder in which multiple videos reside.</b>

Blue cube means that the training is still in progress  
Green cube means that the training is completed  
The skeleton can be found in mmpose/vis_results

In [ ]:
# @title
# #restart runtime before running this code for import MMPose to work
import os
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from IPython.display import clear_output
from moviepy.editor import *
from mmpose.apis import MMPoseInferencer
from tqdm.notebook import tqdm

# Base path
base_path = '/content/ict3104-team01-2023/data/videos'

# Subfolders list
subfolders = [folder for folder in os.listdir(base_path)]

# Collect all .mp4 files from the subfolders with headers
files = [],
for subfolder in subfolders:
    subfolder_path = os.path.join(base_path, subfolder)
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)) and f.endswith('.mp4'):
            files.append(f"{subfolder} - {f}")

# Create a checkbox...
multiple_videos = widgets.Checkbox(
    value=True,
    description='Generate Multiple Videos?',
    disabled=False,
    indent=False
)

# Create a dropdown menu with the folder names
folder_selector = widgets.Dropdown(
    options=subfolders,
    description='Select Folder:',
    disabled=False,
)

# Create a dropdown menu with the file names
file_selector = widgets.Dropdown(
    options=files,
    description='Select Video:',
    disabled=False,
)

visuals_dir = '/content/mmpose/vis_results'
os.makedirs(visuals_dir, exist_ok=True)
inferencer = MMPoseInferencer('human')

# def generate_skeleton(change):
#   folder, filename = file_selector.value.split(" - ", 1)
#   img_path = os.path.join('/content/ict3104-team01-2023/data/videos', folder, filename)
#   result_generator = inferencer(img_path, black_background = True,vis_out_dir='/content/mmpose/vis_results')
#   clear_output(wait=True)
#   print(filename+" selected.")
#   # Initialize tqdm in manual mode
#   pbar = tqdm(total=None, desc="Training progress", unit="item", position=0, leave=True)
#   # Iterate over result_generator and store results
#   results = []
#   for result in result_generator:
#       results.append(result)
#       #Progress bar won't grow as we do not know the total count of iterations
#       pbar.update(1)
#   pbar.close()
#   #Redisplay UI
#   clear_output(wait=True)
#   print(filename+" Skeleton Generated.")
#   display(file_selector)
#   display(gen_button)

def generate(folder, filename):
    img_path = os.path.join('/content/ict3104-team01-2023/data/videos', folder, filename)
    result_generator = inferencer(img_path, black_background = True,vis_out_dir='/content/mmpose/vis_results')
    clear_output(wait=True)
    print(filename+" selected.")
    # Initialize tqdm in manual mode
    pbar = tqdm(total=None, desc="Training progress", unit="item", position=0, leave=True)
    # Iterate over result_generator and store results
    results = []
    for result in result_generator:
        results.append(result)
        #Progress bar won't grow as we do not know the total count of iterations
        pbar.update(1)
    pbar.close()


def generate_skeleton(change):
    clear_output(wait=True)
    if (multiple_videos.value):
        folder_path = os.path.join(base_path, folder_selector.value)
        for filename in os.listdir(folder_path):
              if filename.endswith('.mp4'):
                  folder = folder_selector.value
                  generate(folder, filename)
    else:
        folder, filename = file_selector.value.split(" - ", 1)
        generate(folder, filename)

    #Redisplay UI
    clear_output(wait=True)
    print(filename+" Skeleton Generated.")
    display(file_selector)
    display(gen_button)

# clear_output(wait=True)
# Create a button to play the video
gen_button = widgets.Button(description="Generate MMPose Skeleton")
# Attach the play_video function to the button's on_click event
gen_button.on_click(generate_skeleton)

# Display the dropdown menu and play button
display(multiple_videos)
display(folder_selector)
display(file_selector)
display(gen_button)

##**Testing Section** 📝📝📝
####This section allows you to evaluate a trained model based on the Charades project.
-------------------
  
####How to use (Please Add):
1.
2.
3.

###Load up to 5 pretrained models

In [57]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

folder_path = '/content/ict3104-team01-2023/data/pre_trained_models/'

# List all files in the folder
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Create a layout for the train_file_selector with adjusted width
layout = widgets.Layout(width='350px')  # Adjust width as needed

# Create a multiple-selection widget with the new layout
train_file_selector = widgets.SelectMultiple(
    options=files,
    disabled=False,
    rows=5,  # Number of visible rows
    layout=layout
)

def on_values_change(change):
    if len(train_file_selector.value) > 5:
        # Display a warning message and deselect the last-selected value
        clear_output(wait=True)
        print("You can select up to 5 files only!")
        display(instruction)
        display(train_file_selector)
        # Deselect the last selected option
        train_file_selector.value = train_file_selector.value[:-1]

# Attach the callback function to the value trait of the train_file_selector
train_file_selector.observe(on_values_change, 'value')

# Create an instruction widget using HTML
instruction = widgets.HTML(
    value="<i>Use CTRL or SHIFT to select multiple files.</i><br><i>Select Models:</i>",
    placeholder='',
    description='',
)

# Display the instruction and the multiple-selection widget
display(instruction)
display(train_file_selector)

HTML(value='<i>Use CTRL or SHIFT to select multiple files.</i><br><i>Select Models:</i>', placeholder='')

SelectMultiple(layout=Layout(width='350px'), options=('pose1.yaml', 'default.yaml', 'pose2.yaml'), value=())

###Progress Bar of testing done ()

In [ ]:
from tqdm.notebook import tqdm
# Initialize tqdm in manual mode, pbar will be your progress bar
pbar = tqdm(total=None, desc="Training progress", unit="item", position=0, leave=True)
# Iterate over result_generator and store results
results = []
for result in result_generator:
    results.append(result)
    #Progress bar will increment by 1 every loop, as long as number is incrementing it means that work is being done
    pbar.update(1)
pbar.close()
#Optional step below, you may want to clear the output after the work is done as the text will be left behind and
#the user has to scroll all the way down to see any additional messages u would like to add
clear_output(wait=True)

### Load the models to generate the videos with caption

In [58]:
# This will print the selected files as a tuple
print(train_file_selector.value)
%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
import os
originPath = "/content/ict3104-team01-2023/data/pre_trained_models/"
for config_file in train_file_selector.value:
    # Set the command with the current config file
    print("Working on: " + config_file)
    config_file = originPath + config_file
    command = f'TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="{config_file}"  --skeleton_path="./pose_example/vis_ikun_pose2.mov"'
    os.system(command)


('pose1.yaml', 'default.yaml', 'pose2.yaml')
/content/ict3104-team01-2023/FollowYourPoseTeam1
Working on: pose1.yaml
Working on: default.yaml
Working on: pose2.yaml


###Play videos side by side

In [59]:
import ipywidgets as widgets
from IPython.display import display, HTML, Video
from moviepy.editor import *
from IPython.display import clear_output
from base64 import b64encode
from ipywidgets import Output, GridspecLayout
import os
import glob


def play_video(change):
    clear_output(wait=True)  # clear the current cell output before displaying new video
    if file_selector1.value and file_selector2.value:

            originPath = '/content/GeneratedVideos'
            selected_folder = folder_selector.value  # Get the selected folder name

            video_path1 = os.path.join(originPath, selected_folder, file_selector1.value)
            video_path2 = os.path.join(originPath, selected_folder, file_selector2.value)
            video_clip1 = VideoFileClip(video_path1)
            video_clip2 = VideoFileClip(video_path2)

            # Resize the videos to the desired dimensions
            video_resized1 = video_clip1.resize(newsize=(256, 256))
            video_resized2 = video_clip2.resize(newsize=(256, 256))

            # Create a GridspecLayout for displaying the videos side by side
            grid = GridspecLayout(1, 2)

            out1 = Output()
            out2 = Output()

            with out1:
                display(Video(video_path1, embed=True))
            with out2:
                display(Video(video_path2, embed=True))

            grid[0, 0] = out1
            grid[0, 1] = out2

            display(grid)

            # Display the videos
            display(folder_selector)
            display(update_files_button)
            display(file_selector1)  # re-display the first dropdown
            display(file_selector2)  # re-display the second dropdown
            display(play_button)  # re-display the play button
    else:
        print("Please select two videos")

# Base path
base_path = '/content/GeneratedVideos'

folder_names = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

files = []

# Create a folder selection dropdown
folder_selector = widgets.Dropdown(
    options=folder_names,
    description='Select Folder:',
    disabled=False,
)

# Create a button to update the file options when a folder is selected
update_files_button = widgets.Button(description="Select Folder")

def update_files_button_clicked(b):
    global selected_folder
    selected_folder = os.path.join(base_path, folder_selector.value)
    files = glob.glob(os.path.join(selected_folder, '**/*.mp4'), recursive=True)
    # Extract only the file names without the path
    file_names = [os.path.basename(file) for file in files]
    file_selector1.options = file_names
    file_selector2.options = file_names


update_files_button.on_click(update_files_button_clicked)

# Create two dropdown menus with the file names
file_selector1 = widgets.Dropdown(
    options=files,
    description='Select Video 1:',
    disabled=False,
)

file_selector2 = widgets.Dropdown(
    options=files,
    description='Select Video 2:',
    disabled=False,
)

# Create a button to play the videos
play_button = widgets.Button(description="Select Videos")

# Attach the play_video function to the button's on_click event
play_button.on_click(play_video)

# Display the folder selection dropdown and update button
display(folder_selector)
display(update_files_button)

# Initialize selected_folder
selected_folder = os.path.join(base_path, folder_selector.value)
update_files_button_clicked(None)  # Update file options for the initial folder

# Display the video selection dropdowns and play button
display(file_selector1)
display(file_selector2)
display(play_button)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

Dropdown(description='Select Folder:', index=1, options=('1000-33-2023-10-30 03:41:12.300705', '1000-33-2023-1…

Button(description='Select Folder', style=ButtonStyle())

Dropdown(description='Select Video 1:', index=1, options=('Astronaut on the beach.mp4', 'Astronaut on the beac…

Dropdown(description='Select Video 2:', options=('Astronaut on the beach.mp4', 'Astronaut on the beach_with_ca…

Button(description='Select Videos', style=ButtonStyle())